In [16]:
from pathlib import Path
import os

from google.colab import drive
drive.mount("/content/drive")

# --- EDIT THESE PATHS ONCE ---
DRIVE_CODE_SNAPSHOT = Path("/content/drive/MyDrive/DS_rakuten_colab")
DRIVE_STORE = Path("/content/drive/MyDrive/DS_rakuten_store")
DRIVE_SPLITS_SRC = DRIVE_STORE / "splits"   # expects train_idx.txt / val_idx.txt / test_idx.txt
# ----------------------------

assert DRIVE_CODE_SNAPSHOT.exists(), f"Missing code snapshot: {DRIVE_CODE_SNAPSHOT}"
DRIVE_STORE.mkdir(parents=True, exist_ok=True)

os.environ["DS_RAKUTEN_STORE"] = str(DRIVE_STORE)

print("✓ DRIVE_CODE_SNAPSHOT:", DRIVE_CODE_SNAPSHOT)
print("✓ DRIVE_STORE:", DRIVE_STORE)
print("✓ DRIVE_SPLITS_SRC:", DRIVE_SPLITS_SRC)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✓ DRIVE_CODE_SNAPSHOT: /content/drive/MyDrive/DS_rakuten_colab
✓ DRIVE_STORE: /content/drive/MyDrive/DS_rakuten_store
✓ DRIVE_SPLITS_SRC: /content/drive/MyDrive/DS_rakuten_store/splits


In [17]:
import shutil
import sys
from pathlib import Path

RUNTIME_ROOT = Path("/content/DS_rakuten")

# Clean and copy for deterministic imports
if RUNTIME_ROOT.exists():
    shutil.rmtree(RUNTIME_ROOT)

shutil.copytree(DRIVE_CODE_SNAPSHOT, RUNTIME_ROOT)

sys.path.insert(0, str(RUNTIME_ROOT))

print("✓ Runtime code ready:", RUNTIME_ROOT)
print("✓ sys.path[0]:", sys.path[0])


✓ Runtime code ready: /content/DS_rakuten
✓ sys.path[0]: /content/DS_rakuten


In [18]:
from pathlib import Path
import shutil

runtime_splits_dir = Path("/content/DS_rakuten/data/splits")
runtime_splits_dir.mkdir(parents=True, exist_ok=True)

# Copy txt files from Drive persistent store into /content runtime repo
src_files = ["train_idx.txt", "val_idx.txt", "test_idx.txt"]
for fn in src_files:
    src = DRIVE_SPLITS_SRC / fn
    dst = runtime_splits_dir / fn
    assert src.exists(), f"Missing split file in Drive: {src}"
    shutil.copy2(src, dst)

print("✓ Splits synced to:", runtime_splits_dir)
print("✓ Contents:", list(runtime_splits_dir.glob("*.txt"))[:10])


✓ Splits synced to: /content/DS_rakuten/data/splits
✓ Contents: [PosixPath('/content/DS_rakuten/data/splits/test_idx.txt'), PosixPath('/content/DS_rakuten/data/splits/val_idx.txt'), PosixPath('/content/DS_rakuten/data/splits/train_idx.txt')]


In [20]:
from pathlib import Path

IMAGE_FILE_ID = "15ZkS0iTQ7j3mHpxil4mABlXwP-jAN_zi"

BASE_DIR = Path("/content/images")
TMP_DIR = Path("/content/tmp")
ZIP_PATH = TMP_DIR / "images.zip"

BASE_DIR.mkdir(parents=True, exist_ok=True)
TMP_DIR.mkdir(parents=True, exist_ok=True)

if not ZIP_PATH.exists():
    print("Downloading images zip...")
    !gdown --id $IMAGE_FILE_ID -O {str(ZIP_PATH)}
else:
    print("Zip already present:", ZIP_PATH)

print("Unzipping images...")
!unzip -q -o {str(ZIP_PATH)} -d {str(BASE_DIR)}

def count_jpgs(p: Path, limit: int = 2000) -> int:
    if not p.exists():
        return 0
    n = 0
    for _ in p.rglob("*.jpg"):
        n += 1
        if n >= limit:
            break
    return n

# Common candidates
candidates = [
    BASE_DIR / "images" / "image_train",
    BASE_DIR / "image_train",
    BASE_DIR / "images" / "images" / "image_train",
]

best = None
best_count = 0
for c in candidates:
    n = count_jpgs(c)
    if n > best_count:
        best, best_count = c, n

# Fallback: search any folder named image_train
if best_count == 0:
    for c in BASE_DIR.rglob("image_train"):
        if c.is_dir():
            n = count_jpgs(c)
            if n > best_count:
                best, best_count = c, n

assert best is not None and best_count > 0, (
    "Could not find an image_train directory with jpg files under /content/images. "
    "Check zip content and unzip path."
)

IMG_ROOT = best
sample_jpg = next(IMG_ROOT.rglob("*.jpg"))

print("✓ IMG_ROOT detected:", IMG_ROOT)
print("✓ sample jpg:", sample_jpg)


Zip already present: /content/tmp/images.zip
Unzipping images...
✓ IMG_ROOT detected: /content/images/images/image_train
✓ sample jpg: /content/images/images/image_train/image_1010030825_product_443748930.jpg


In [21]:
from src.data.image_dataset import RakutenImageDataset
from src.train.image_resnet50 import ResNet50Config, run_resnet50_colab

print("✓ RakutenImageDataset:", RakutenImageDataset)
print("✓ ResNet50Config:", ResNet50Config)
print("✓ run_resnet50_colab:", run_resnet50_colab)


✓ RakutenImageDataset: <class 'src.data.image_dataset.RakutenImageDataset'>
✓ ResNet50Config: <class 'src.train.image_resnet50.ResNet50Config'>
✓ run_resnet50_colab: <function run_resnet50_colab at 0x7dbbe07dd940>


In [22]:
from src.data.split_manager import load_splits, split_signature

splits = load_splits(verbose=True)
sig = split_signature(splits)

print("✓ signature:", sig)
print({k: len(v) for k, v in splits.items()})


[split_manager] Loading canonical splits from /content/DS_rakuten/data/splits
✓ signature: cf53f8eb169b3531
{'train_idx': 61351, 'val_idx': 10827, 'test_idx': 12738}


In [23]:
import os
from pathlib import Path

STORE = Path(os.environ["DS_RAKUTEN_STORE"])

cfg = ResNet50Config(
    raw_dir="/content/drive/MyDrive/DS_rakuten_store/data_raw",
    img_dir=str(IMG_ROOT),  # must be /content local disk for speed
    out_dir=str(STORE / "artifacts" / "exports"),
    ckpt_dir=str(STORE / "checkpoints" / "image_resnet50"),

    img_size=224,
    batch_size=512,
    num_workers=8,
    num_epochs=30,
    lr=3e-4,

    use_amp=True,
    label_smoothing=0.1,
    dropout_rate=0.3,

    model_name="resnet50_rerun_canonical",
    export_split="val",
)

result = run_resnet50_colab(cfg)

print("EXPORT:", result["export_result"])
print("VERIFY:", result["verify_metadata"])
print("probs_shape:", result["probs_shape"])
print("best_val_f1:", result["best_val_f1"])


[load_data_colab] raw_dir: /content/drive/MyDrive/DS_rakuten_store/data_raw
[load_data_colab] img_root: /content/images/images/image_train
[load_data_colab] X: /content/drive/MyDrive/DS_rakuten_store/data_raw/X_train_update.csv
[load_data_colab] Y: /content/drive/MyDrive/DS_rakuten_store/data_raw/Y_train_CVw08PX.csv
[split_manager] Loading canonical splits from /content/DS_rakuten/data/splits


Epoch 1/30 | train_loss=1.9456 train_f1=0.4336 | val_loss=1.6791 val_f1=0.5497 | lr=3.00e-04


Epoch 2/30 | train_loss=1.5128 train_f1=0.6264 | val_loss=1.5648 val_f1=0.6066 | lr=3.00e-04


Epoch 3/30 | train_loss=1.3480 train_f1=0.6978 | val_loss=1.5659 val_f1=0.6223 | lr=3.00e-04


Epoch 4/30 | train_loss=1.2131 train_f1=0.7581 | val_loss=1.5777 val_f1=0.6332 | lr=3.00e-04


...

Epoch 27/30 | train_loss=0.6889 train_f1=0.9910 | val_loss=1.6456 val_f1=0.6451 | lr=3.00e-08


Epoch 28/30 | train_loss=0.6893 train_f1=0.9896 | val_loss=1.6477 val_f1=0.6445 | lr=3.00e-08


Epoch 29/30 | train_loss=0.6884 train_f1=0.9901 | val_loss=1.6452 val_f1=0.6438 | lr=3.00e-08


Epoch 30/30 | train_loss=0.6892 train_f1=0.9897 | val_loss=1.6452 val_f1=0.6458 | lr=3.00e-09


[OK] Exported model=resnet50_rerun_canonical split=val npz=/content/drive/MyDrive/DS_rakuten_store/artifacts/exports/resnet50_rerun_canonical/val.npz sig=cf53f8eb169b3531 fp=cdfa70b13f7390e6 n=10827
EXPORT: {'npz_path': '/content/drive/MyDrive/DS_rakuten_store/artifacts/exports/resnet50_rerun_canonical/val.npz', 'meta_json_path': '/content/drive/MyDrive/DS_rakuten_store/artifacts/exports/resnet50_rerun_canonical/val_meta.json', 'classes_fp': 'cdfa70b13f7390e6', 'split_signature': 'cf53f8eb169b3531', 'num_samples': 10827}
VERIFY: {'model_name': 'resnet50_rerun_canonical', 'split_name': 'val', 'split_signature': 'cf53f8eb169b3531', 'classes_fp': 'cdfa70b13f7390e6', 'num_classes': 27, 'num_samples': 10827, 'has_y_true': True, 'probs_shape': [10827, 27], 'probs_dtype': 'float32', 'created_at': '2026-01-08T18:14:40.324946', 'extra': {'source': 'src/train/image_resnet50.py', 'model_architecture': 'torchvision.resnet50', 'img_dir': '/content/images/images/image_train', 'img_size': 224, 'batch

In [24]:
import os
from pathlib import Path

STORE = Path(os.environ["DS_RAKUTEN_STORE"])
export_dir = STORE / "artifacts" / "exports" / "resnet50_rerun_canonical"

print("Export dir:", export_dir)
print("Contents:", [p.name for p in export_dir.glob("*")])

assert (export_dir / "val.npz").exists(), "Missing val.npz"
assert (export_dir / "val_meta.json").exists(), "Missing val_meta.json"
print("✓ Export files exist.")


Export dir: /content/drive/MyDrive/DS_rakuten_store/artifacts/exports/resnet50_rerun_canonical
Contents: ['val.npz', 'val_meta.json']
✓ Export files exist.


In [25]:
!python -m apps.image_app.scripts.validate_exports -split val -strict


/usr/bin/python3: Error while finding module specification for 'apps.image_app.scripts.validate_exports' (ModuleNotFoundError: No module named 'apps')


In [26]:
import json
from pathlib import Path
import os

STORE = Path(os.environ["DS_RAKUTEN_STORE"])
meta_path = STORE / "artifacts" / "exports" / "resnet50_rerun_canonical" / "val_meta.json"

meta = json.loads(meta_path.read_text())
keys = [
    "model_name", "split_name", "split_signature",
    "classes_fp", "num_samples", "probs_shape"
]
for k in keys:
    print(f"{k}: {meta.get(k)}")


model_name: resnet50_rerun_canonical
split_name: val
split_signature: cf53f8eb169b3531
classes_fp: cdfa70b13f7390e6
num_samples: 10827
probs_shape: [10827, 27]


In [27]:
import shutil
from pathlib import Path
from src.export.model_exporter import load_predictions
from src.data.label_mapping import CANONICAL_CLASSES_FP
from src.data.split_manager import load_splits, split_signature

splits = load_splits(verbose=False)
sig = split_signature(splits)

CACHE = Path("/content/cache_exports")
CACHE.mkdir(parents=True, exist_ok=True)

export_result = result["export_result"]
npz_src = Path(export_result["npz_path"])
meta_src = npz_src.with_name(npz_src.stem + "_meta.json")

npz_local = CACHE / npz_src.name
meta_local = CACHE / meta_src.name

# Copy both files (npz + meta)
if (not npz_local.exists()) or (npz_local.stat().st_size != npz_src.stat().st_size):
    shutil.copy2(npz_src, npz_local)

if (not meta_local.exists()) or (meta_local.stat().st_size != meta_src.stat().st_size):
    shutil.copy2(meta_src, meta_local)

loaded = load_predictions(
    npz_path=str(npz_local),
    verify_split_signature=sig,
    verify_classes_fp=CANONICAL_CLASSES_FP,
    require_y_true=True,
)

print("✓ loaded ok")
print("model:", loaded["metadata"]["model_name"])
print("split:", loaded["metadata"]["split_name"])
print("sig:", loaded["metadata"]["split_signature"])
print("fp:", loaded["metadata"]["classes_fp"])
print("probs:", loaded["probs"].shape)


✓ loaded ok
model: resnet50_rerun_canonical
split: val
sig: cf53f8eb169b3531
fp: cdfa70b13f7390e6
probs: (10827, 27)


In [28]:
import os
from pathlib import Path

STORE = Path(os.environ["DS_RAKUTEN_STORE"])
export_dir = STORE / "artifacts" / "exports" / "resnet50_rerun_canonical"

print("Export dir:", export_dir)
print("Files:", [p.name for p in export_dir.glob("*")])

assert (export_dir / "val.npz").exists(), "Missing val.npz"
assert (export_dir / "val_meta.json").exists(), "Missing val_meta.json"
print("✓ Export files exist")


Export dir: /content/drive/MyDrive/DS_rakuten_store/artifacts/exports/resnet50_rerun_canonical
Files: ['val.npz', 'val_meta.json']
✓ Export files exist
